In [ ]:
!pip install transformers
!pip install evaluate

In [3]:
# !pip install SentencePiece

In [3]:
import pandas as pd

df_qafinal = pd.read_csv('/content/drive/MyDrive/Booksummary/QA_Final.csv')

df_prompt_qa = pd.read_csv("/content/drive/MyDrive/Booksummary/Question_Prompts_with_QA.csv")

df_prompt_qa  = df_prompt_qa.loc[df_prompt_qa.QA.notna()][['Section_Id','Context_Id','Prompt']]
df_prompt_qa['Context'] = df_prompt_qa['Prompt'].str.replace('Text: ', "|").str.replace('Question: \n', "|").str.split("|").str[1].str.strip()

df_prompt_qa = df_prompt_qa[['Section_Id','Context_Id', 'Context']]

df_qafinal = df_qafinal.merge(df_prompt_qa, left_on="ContextId", right_on="Context_Id", how="left")
df_qafinal['Question'] = df_qafinal.Question.str.replace('Question: ','')
df_qafinal['Answer'] = df_qafinal.Answer.str.replace('Answer: ','')
data = df_qafinal[['QuesId','Question','Answer','Context']]
data.rename(columns={"Question":"question","Answer":"answer","Context":"context"},inplace=True)
data

<ipython-input-3-4a21603e3de5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Question":"question","Answer":"answer","Context":"context"},inplace=True)


,QuesId,question,answer,context
0,1,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
1,2,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...",THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
2,3,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...",THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
3,4,How will the universe be affected at the end o...,At the end of the world and the expiration of ...,"And what is seen in the universe, whether anim..."
4,5,"How many generations of Devas were there, and ...",The generation of Devas consisted of thirty-th...,"And what is seen in the universe, whether anim..."
...,...,...,...,...
1114,1115,How does Agni respond to Drona's request?,Agni responds positively to Drona's request an...,"""Drona said, 'O lord of the universe, growing ..."
1115,1116,"Who is Jarita referring to as the ""eldest"" and...",Jarita is referring to the eldest and next chi...,"""Jarita then said, 'What hast thou to do with ..."
1116,1117,What does Mandapala say about co-wives and cla...,Mandapala states that co-wives and clandestine...,"""Jarita then said, 'What hast thou to do with ..."
1117,1118,Who does Mandapala compare Jarita's jealousy to?,Mandapala compares Jarita's jealousy to Arundh...,"""Jarita then said, 'What hast thou to do with ..."


In [4]:
Question_Len = max(len(ques) for ques in data.question)
Answer_Len = max(len(ans) for ans in data.answer)
Question_Len,Answer_Len

(144, 890)

In [5]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

import warnings
warnings.filterwarnings("ignore")

In [49]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True, return_tensors='pt')
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True, return_tensors='pt')

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [7]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [23]:
# TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
# MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)


MODEL = AutoModelForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
TOKENIZER = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# Set the padding token to eos_token
TOKENIZER.pad_token = TOKENIZER.eos_token

MODEL.to(DEVICE)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)

In [24]:
Q_LEN = Question_Len # 256   # Question Length
T_LEN = Answer_Len #32  # Target Length
BATCH_SIZE = 2
DEVICE = "cuda:0"

In [25]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader   = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [34]:
from transformers import BertTokenizer, BertModel
TOKENIZER = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
MODEL = BertModel.from_pretrained("bert-base-multilingual-cased")

In [50]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(2):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Training batches:   0%|          | 0/448 [00:00<?, ?it/s]


TypeError: ignored

In [14]:
MODEL.save_pretrained("/content/drive/MyDrive/Booksummary/t5qa_model")
TOKENIZER.save_pretrained("/content/drive/MyDrive/Booksummary/t5qa_tokenizer")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

In [15]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [16]:
i=10
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Context: 
 Pandu, after having subdued many countries by his wisdom and prowess, took up his abode with the Munis in a certain forest as a sportsman, where he brought upon himself a very severe misfortune for having killed a stag coupling with its mate, which served as a warning for the conduct of the princes of his house as long as they lived. Their mothers, in order that the ordinances of the law might be fulfilled, admitted as substitutes to their embraces the gods Dharma, Vayu, Sakra, and the divinities the twin Aswins. And when their offspring grew up, under the care of their two mothers, in the society of ascetics, in the midst of sacred groves and holy recluse-abodes of religious men, they were conducted by Rishis into the presence of Dhritarashtra and his sons, following as students in the habit of Brahmacharis, having their hair tied in knots on their heads. 'These our pupils', said they, 'are as your sons, your brothers, and your friends; they are Pandavas.' Saying this, the 

{'Reference Answer: ': 'The principal men were pleased with the virtues and qualities displayed by the Pandavas. They respected Yudhishthira for his purity, Arjuna for his courage, Kunti for her submissive attention to her superiors, and Nakula and Sahadeva for their humility. The people rejoiced in the heroic virtues of the Pandavas.',
 'Predicted Answer: ': 'The principal men and the people perceived the Pandavas as being a great and powerful force,',
 'BLEU Score: ': {'google_bleu': 0.06542056074766354}}

In [17]:
i=100
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Context: 
 "On the other hand, the Danavas, white as the clouds from which the rain hath dropped, possessing great strength and bold hearts, ascended the sky, and by hurling down thousands of mountains, continually harassed the gods. And those dreadful mountains, like masses of clouds, with their trees and flat tops, falling from the sky, collided with one another and produced a tremendous roar. And when thousands of warriors shouted without intermission in the field of battle and mountains with the woods thereon began to fall around, the earth with her forests trembled. Then the divine Nara appeared at the scene of the dreadful conflict between the Asuras and the Ganas (the followers of Rudra), and reducing to dust those rocks by means of his gold-headed arrows, he covered the heavens with dust. Thus discomfited by the gods, and seeing the furious discus scouring the fields of heaven like a blazing flame, the mighty Danavas entered the bowels of the earth, while others plunged into th

{'Reference Answer: ': 'The Danavas hurled down thousands of mountains from the sky to harass the gods according to the text.',
 'Predicted Answer: ': 'The Danavas hurled down thousands of mountains from the sky to harass the gods',
 'BLEU Score: ': {'google_bleu': 0.7142857142857143}}

In [19]:
i=201
pred_ans = predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Context: 
 "Sauti continued, 'The tree was reduced to ashes by the poison of that king of snakes. But taking up those ashes, Kasyapa spoke these words. 'O king of snakes, behold the power of my knowledge as applied to this lord of the forest! O snake, under thy very nose I shall revive it.' And then that best of Brahmanas, the illustrious and learned Kasyapa, revived, by his vidya, that tree which had been reduced to a heap of ashes. And first he created the sprout, then he furnished it with two leaves, and then he made the stem, and then the branches, and then the full-grown tree with leaves and all. And Takshaka, seeing the tree revived by the illustrious Kasyapa, said unto him, 'It is not wonderful in thee that thou shouldst destroy my poison or that of any one else like myself. O thou whose wealth is asceticism, desirous of what wealth, goest thou thither? The reward thou hopest to have from that best of monarchs, even I will give thee, however difficult it may be to obtain it. Dec

In [26]:
for k,v in pred_ans.items():
  print (k)

Reference Answer: 
Predicted Answer: 
BLEU Score: 


In [ ]:
pred_answers=[]
bleu_score=[]
for i in range(len(data)):
  pred_ans = predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)
  pred_answers.append(pred_ans['Predicted Answer: '])
  bleu_score.append(pred_ans['BLEU Score: '])


In [31]:
predict_answer(data.iloc[i]['context'],data.iloc[i]['question'],data.iloc[i]['answer'],)

Context: 
 THE MAHABHARATA ADI PARVA Section I
Om! Having bowed down to Narayana and Nara, the most exalted male being, and also to the goddess Saraswati, must the word Jaya be uttered.
Ugrasrava, the son of Lomaharshana, surnamed Sauti, well-versed in the Puranas, bending with humility, one day approached the great sages of rigid vows, sitting at their ease, who had attended the twelve years' sacrifice of Saunaka, surnamed Kulapati, in the forest of Naimisha. Those ascetics, wishing to hear his wonderful narrations, presently began to address him who had thus arrived at that recluse abode of the inhabitants of the forest of Naimisha. Having been entertained with due respect by those holy men, he saluted those Munis (sages) with joined palms, even all of them, and inquired about the progress of their asceticism. Then all the ascetics being again seated, the son of Lomaharshana humbly occupied the seat that was assigned to him. Seeing that he was comfortably seated, and recovered from f

{'Reference Answer: ': 'The Mahabharata was composed by Krishna-Dwaipayana (Vyasa) and recited by Vaisampayana at the snake-sacrifice of Raja Janamejaya.',
 'Predicted Answer: ': 'The Mahabharata was composed by Ugrasrava, the son of Lom',
 'BLEU Score: ': {'google_bleu': 0.21621621621621623}}

In [52]:
df1 = pd.DataFrame({"pred_answer": pred_answers, "bleu_score":bleu_score})
df1.index = data.index

val_data_index = val_data.index

df1['train_data']=False

for i in val_data_index:
  df1.loc[i,"train_data"]=True

In [55]:
df1.to_csv('/content/drive/MyDrive/Booksummary/t5predicted_ans.csv')

In [54]:
df1

,pred_answer,bleu_score,train_data
0,"The Mahabharata was composed by Ugrasrava, the...",{'google_bleu': 0.21621621621621623},False
1,"The Mahabharata is a sacred and sacred book, w...",{'google_bleu': 0.09340659340659341},False
2,The creation of the world began according to t...,{'google_bleu': 0.02608695652173913},False
3,The universe will be affected at the end of th...,{'google_bleu': 0.19801980198019803},True
4,The generation of Devas was thirty-three thous...,{'google_bleu': 0.06204379562043796},False
...,...,...,...
1114,Agni responds to Drona's request by saying tha...,{'google_bleu': 0.16216216216216217},False
1115,"Jarita refers to the ""eldest"" and ""next"" among...",{'google_bleu': 0.1111111111111111},False
1116,Mandapala says that co-wives and clandestine l...,{'google_bleu': 0.14},True
1117,Mandapala compares Jarita's jealousy to the je...,{'google_bleu': 0.14912280701754385},False
